In [1]:
cd /private/home/dpf/projects/codex

/private/home/dpf/projects/codex


In [2]:
from models import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from he import make_parser, HUMAN_EVAL_STOP_WORDS
import shlex
import pprint

In [5]:
args = parser.parse_args(shlex.split("--model_name /checkpoint/dpf/models/lm-1.3B-ourtok/checkpoint_last_consolidated.pt --num_candidates_generated 1 --num_candidates_evaluated 1 --batch_size 3 --temperature 0.2 --top_p 0.95 --output_filename expts/he/lm-1.3B-ourtok_last_ncg-1_temp-0.2/prompt_pyfile_normalized/samples.jsonl --response_filename expts/he/lm-1.3B-ourtok_last_ncg-1_temp-0.2/prompt_pyfile_normalized/responses.pkl --prompt_prefix '<| file ext=.py |>'"))

NameError: name 'parser' is not defined

In [ ]:
pprint.pprint(vars(args))

In [ ]:
model = make_model(args, args.model_name, args.tokenizer_name, prompt_prefix=args.prompt_prefix)

In [6]:
from human_eval.data import read_problems

In [7]:
problems = read_problems()

In [8]:
task_id, problem = next(iter(problems.items()))

In [33]:
prompt = problem['prompt']

In [36]:
completions, response = model.rank_completions(
    prompt, HUMAN_EVAL_STOP_WORDS,
    max_tokens=450,
    n=args.num_candidates_generated,
    cached_response=None,
    scoring=args.candidate_scoring,
    temperature=args.temperature,
    top_p=args.top_p,
)

In [38]:
problem

{'task_id': 'HumanEval/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'entry_point': 'has_close_elements',
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9,

In [37]:
completions

[{'text': '    return any(abs(x - y) < threshold for x, y in zip(numbers, numbers[1:]))\n\n',
  'logprobs': {'token_logprobs': [-1.502655029296875,
    -1.01348876953125,
    -0.00750732421875,
    -0.46185302734375,
    0.0,
    -0.00640869140625,
    0.0,
    -0.033660888671875,
    0.0,
    0.0,
    0.0,
    0.0,
    -0.040435791015625,
    0.0,
    0.0,
    0.0,
    -0.065826416015625],
   'tokens': ['    return ',
    'any(',
    'abs(',
    'x - ',
    'y',
    ') < ',
    'threshold ',
    'for ',
    'x, ',
    'y in ',
    'zip(',
    'number',
    's, ',
    'number',
    's[1:',
    ']))',
    '\n\n']}}]

In [42]:
prompt

'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n'

In [41]:
completions

[{'text': '    return any(abs(x - y) < threshold for x, y in zip(numbers, numbers[1:]))\n\n',
  'logprobs': {'token_logprobs': [-1.502655029296875,
    -1.01348876953125,
    -0.00750732421875,
    -0.46185302734375,
    0.0,
    -0.00640869140625,
    0.0,
    -0.033660888671875,
    0.0,
    0.0,
    0.0,
    0.0,
    -0.040435791015625,
    0.0,
    0.0,
    0.0,
    -0.065826416015625],
   'tokens': ['    return ',
    'any(',
    'abs(',
    'x - ',
    'y',
    ') < ',
    'threshold ',
    'for ',
    'x, ',
    'y in ',
    'zip(',
    'number',
    's, ',
    'number',
    's[1:',
    ']))',
    '\n\n']}}]

In [67]:
enc = model._encode("def ")

In [68]:
enc

tensor([380,   2])

In [95]:
greedy = model.lm_model.generate([enc], sampling=True, beam=1, sampling_topk=1, temperature=0.6)[0][0]

In [96]:
greedy['positional_scores'].sum()

tensor(-73.9279, device='cuda:0')

In [97]:
model._decode(greedy['tokens'])

'def f():\n    return 1\n</code>\n<|/ a dscore=0 |>\n<| c |>\nThanks for your answer. I am not able to understand how this works. Can you please explain it a little bit more?\n<|/ c |>\n<| c |>\n`f` returns a function which takes no arguments and returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f`

In [98]:
len(greedy['tokens'])

451

In [108]:
scores = model.lm_model.score(model._decode(greedy['tokens']))

In [100]:
len(scores['tokens'])

450

In [101]:
len(scores['positional_scores'])

450

In [102]:
greedy['positional_scores'][:10]

tensor([-2.8652e+01, -2.8209e+00, -5.5077e-01, -5.6534e-03, -9.8863e-01,
        -1.1909e+00, -4.0000e-01, -4.8457e-01, -1.0490e-03, -3.5089e-01],
       device='cuda:0')

In [103]:
greedy['positional_scores'].sum()

tensor(-73.9279, device='cuda:0')

In [109]:
scores['positional_scores'][:10]

tensor([-1.7463e+01, -4.5906e+00, -1.7478e+00, -9.2374e-02, -2.0834e+00,
        -2.5510e+00, -6.3093e-01, -1.2731e+00, -1.6337e-02, -1.7059e+00],
       device='cuda:0')

In [105]:
scores['positional_scores'].sum()

tensor(-113.3715, device='cuda:0')

In [74]:
model._decode(scores['tokens'])

'def f():\n    return 1\n</code>\n<|/ a dscore=0 |>\n<| c |>\nThanks for your answer. I am not able to understand how this works. Can you please explain it a little bit more?\n<|/ c |>\n<| c |>\n`f` returns a function which takes no arguments and returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f` which returns `1`. `f` then calls `f`

In [77]:
greedy[0][0]['positional_scores'].sum() / len(greedy[0][0]['positional_scores'])

tensor(-0.2514, device='cuda:0')

In [45]:
model.lm_model.score([prompt])

[{'tokens': tensor([48158,   536,   380,  4732, 14180,  1375,   484,  1726, 14641, 37417,
           6342,  8660, 18567,   205,  7749, 13161,   366,  2649,  2433,  1726,
            463,   937,  2193,  3510, 34254,  1304,   374,  1222,  2065,  2370,
          20711,   205,   264,  2649, 13349,   696,   205,  2296,  4732, 14180,
           1375,  8877,  6280, 22672,  1201, 13102, 12444,   205,   264,   690,
            205,  2296,  4732, 14180,  1375,  8877,  6280,   845,  3554, 41762,
           2409,   546, 41091,   845,   589,  3024,  2644,   205, 28660,   205,
            768,   205], device='cuda:0'),
  'score': tensor(-2.1254, device='cuda:0'),
  'attention': None,
  'alignment': None,
  'positional_scores': tensor([-2.0640e+01, -1.5026e+00, -2.6657e+00, -6.1961e+00, -7.5587e+00,
          -5.9465e+00, -1.9377e-01, -6.1983e+00, -1.4929e-01, -3.9327e+00,
          -3.7841e+00, -2.3243e-01, -8.7383e-01, -1.0815e-02, -4.0083e+00,
          -2.0688e+00, -9.2305e+00, -2.8441e+00, -1.64

In [6]:
### INFILLS

In [9]:
import infill_evaluation

In [10]:
from he import generate_he_infill_problems

In [11]:
from models import TruncationParameters

In [12]:
infill_parser = infill_evaluation.make_parser()

In [65]:
infill_args = infill_parser.parse_args(shlex.split("--model_path /checkpoint/dpf/models/cm-6B-armen/cm-6B-ourtok/best.pt \
  --tokenizer_name gpt2_pretokenization_newlines_only  \
  --candidate_scoring random \
  --batch_size 10 \
  --eval_type one_line \
  --truncation_heuristics num_lines suffix \
  --temperature 0.2 \
  --num_candidates 1 \
  --bidirectional_generation \
  "))

In [66]:
pprint.pprint(vars(infill_args))

{'batch_size': 10,
 'bidirectional_generation': True,
 'bidirectional_scoring': False,
 'candidate_scoring': 'random',
 'eval_type': 'one_line',
 'evaluate_only': False,
 'max_tokens': None,
 'model_path': '/checkpoint/dpf/models/cm-6B-armen/cm-6B-ourtok/best.pt',
 'num_candidates': 1,
 'prompt_prefix': None,
 'result_base_path': None,
 'temperature': 0.2,
 'tokenizer_name': 'gpt2_pretokenization_newlines_only',
 'top_p': 0.95,
 'truncation_heuristics': ['num_lines', 'suffix'],
 'unnormalized': False}


In [64]:
# no caching
# model = make_model(infill_args, infill_args.model_path, infill_args.tokenizer_name, prompt_prefix=infill_args.prompt_prefix)
# with caching
model = make_model(infill_args, infill_args.model_path, infill_args.tokenizer_name, prompt_prefix=infill_args.prompt_prefix, cached_model=model.lm_model)

model_root_dir: /checkpoint/dpf/models/cm-6B-armen/cm-6B-ourtok
model_basename: best.pt


[autoreload of models failed: Traceback (most recent call last):
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
TypeError: __class__ assignm

In [59]:
model

In [19]:
infill_problems = generate_he_infill_problems(infill_args, eval_type="one_line")

In [20]:
infill_problems = list(infill_problems)

In [21]:
task_id, problems = infill_problems[0]

In [22]:
task_id

'HumanEval/0'

In [23]:
problem = problems[0]

In [24]:
problem

{'task_id': 'HumanEval/0',
 'num_before': 0,
 'num_after': 7,
 'missing_lines': '    for idx, elem in enumerate(numbers):',
 'prompt_parts': ['from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
  '        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False'],
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    re

In [25]:
prompt_parts = problem["prompt_parts"]
prefix, suffix = prompt_parts

In [26]:
from models import TruncationParameters

In [67]:
truncation_parameters = [TruncationParameters.from_heuristics(infill_args.truncation_heuristics, problem["missing_lines"], suffix)]

In [69]:
kwargs = dict(
    verbose=False, n=infill_args.num_candidates,
    bidirectional_generation=infill_args.bidirectional_generation,
    bidirectional_scoring=infill_args.bidirectional_scoring,
    truncation_parameters=truncation_parameters,
    scoring=infill_args.candidate_scoring,
)
# if args.temperature == 0.0:
#     # kwargs.update(sampling=False)
# else:
kwargs.update(sampling=True, top_p=infill_args.top_p, temperature=infill_args.temperature)
# kwargs.update(sampling=True, top_p=infill_args.top_p, temperature=0.8)
# kwargs.update(max_tokens=30)

In [70]:
kwargs

{'verbose': False,
 'n': 1,
 'bidirectional_generation': True,
 'bidirectional_scoring': False,
 'truncation_parameters': [TruncationParameters(max_num_lines=1, suffix='        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False')],
 'scoring': 'random',
 'sampling': True,
 'top_p': 0.95,
 'temperature': 0.2}

In [71]:
model.batch_size = 5

In [72]:
sorted_choices, response = model.rank_infills(prompt_parts, **kwargs)

In [73]:
response

{'prompt_parts': ['from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
  '        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False'],
 'choices': [{'complete': ['from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
    ' 

In [74]:
sorted_choices[0].keys()

dict_keys(['complete', 'infills', 'infills_untruncated', 'logprobs'])

In [75]:
sorted_choices[0]

{'complete': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    numbers = sorted(numbers)\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False',
 'infills': ['    numbers = sorted(numbers)'],
 'infills_untruncated': ['    numbers = sorted(numbers)\n    for idx, elem in enumerate(numbers):\n        if idx != 0:\n            distance = abs(elem - numbers[idx - 1])\n            if distance < threshold:\n                return True\n\n    return False\n\n\ndef has_close_elements_list(numbers: List[float], threshold: floa

In [76]:
def truncate_overlap(infill, suffix, minimum_num_characters=None, minimum_num_suffix_lines=1):
    if minimum_num_characters is None:
        non_empty_suffix_lines = [l.strip() for l in suffix.strip("\n") if l.strip()]
        minimum_num_characters = sum(len(l) for l in non_empty_suffix_lines[:minimum_num_suffix_lines])
    for i in range(len(infill), minimum_num_characters, -1):
        if infill[-i:] == suffix[:i]:
            return infill[:-i]
    return infill

In [59]:
pdb on

Automatic pdb calling has been turned ON


In [63]:
model.score_text([sorted_choices[6]['complete']])

[-174.8389892578125]

In [66]:
model.score_text([choice['complete'] for choice in sorted_choices], 'mean')

[-1.5474575757980347,
 -1.5674219131469727,
 -1.5674219131469727,
 -1.5474575757980347,
 -1.5474575757980347,
 -1.5564860105514526,
 -1.5474575757980347,
 -1.5564860105514526,
 -1.5671440362930298,
 -1.5671440362930298]

In [79]:
choice['complete']

'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    numbers.sort()\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False'

In [65]:
[choice['infills'] for choice in sorted_choices]

[['                if distance < threshold:'],
 ['                if distance > threshold:'],
 ['                if distance > threshold:'],
 ['                if distance < threshold:'],
 ['                if distance < threshold:'],
 ['                if distance <= threshold:'],
 ['                if distance < threshold:'],
 ['                if distance <= threshold:'],
 ['                if distance > threshold:'],
 ['                if distance > threshold:']]

In [30]:
from models import OpenAIModel

In [78]:
codex = OpenAIModel()

[autoreload of models failed: Traceback (most recent call last):
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/private/home/dpf/.conda/envs/fairseq-20210318-htlm_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
TypeError: __class__ assignm

In [83]:
import openai
from secret import API_KEY
openai.api_key = API_KEY

In [85]:
response = openai.Completion.create(engine="davinci-codex", prompt=choice['complete'], logprobs=1, max_tokens=0, temperature=1.0, echo=True)

In [87]:
choice = response['choices'][0]

In [92]:
sum(choice['logprobs']['token_logprobs'][1:])

-171.7579190418569